# For EBAY spider

In [1]:
import logging
logging.basicConfig(level=logging.DEBUG)

import requests
from bs4 import BeautifulSoup as soup
import os.path
import json
from datetime import datetime
from datetime import timedelta
import time
import os
import re
import imgkit


In [2]:
from PIL import Image
from io import BytesIO
def _download_image(img_url, dest):
    response = requests.get(img_url)
    MAX_SIZE = 256, 256
    img = Image.open(BytesIO(response.content))
    img = img.convert("RGB") # to solve OSError: cannot write mode RGBA as JPEG
    img.thumbnail(MAX_SIZE, Image.ANTIALIAS)
    img.save(dest)

In [3]:
headers = {
        'User-Agent': 'Mozilla/5.0 (iPhone; CPU iPhone OS 13_2_3 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.0.3 Mobile/15E148 Safari/604.1',
        'Connection' : 'keep-alive',
        'Cache-Control' : 'max-age=0',
        'Upgrade-Insecure-Requests' : '1',
        'Accept' : 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'Accept-Encoding' : 'gzip, deflate, br',
        'Accept-Language' : 'en-US,en;q=0.9,vi;q=0.8',
    }

## For search query

In [4]:
# ending soon and page =200
search_url = "https://www.ebay.de/sch/i.html?_from=R40&_nkw=apple&LH_TitleDesc=1&LH_Auction=1&_sop=1&_ipg=200"

# # Best results and page = 200
# search_url = "https://www.ebay.de/sch/i.html?_from=R40&_nkw=apple&LH_TitleDesc=1&LH_Auction=1&_sop=12&_ipg=200"

In [5]:
response = requests.get(search_url, headers=headers)
with open("myfile.html", "wb") as f: 
    # Writing data to a file 
    f.write(response.content) 

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.ebay.de:443
DEBUG:urllib3.connectionpool:https://www.ebay.de:443 "GET /sch/i.html?_from=R40&_nkw=apple&LH_TitleDesc=1&LH_Auction=1&_sop=1&_ipg=200 HTTP/1.1" 200 None


In [6]:
# CRAWL_FREQUENCE_IN_SECONDS = 3
# count = 0
# crawl_next_call = time.time()
# while True:
#     status_code = requests.get(search_url, headers=headers)
#     count += 1
#     crawl_next_call = crawl_next_call + CRAWL_FREQUENCE_IN_SECONDS
#     sleep_duration = max(crawl_next_call - time.time(), 0.001)
#     logging.debug("Turn {}.".format(count))
#     logging.debug("Time to next crawling turn = {}.".format(crawl_next_call - time.time()))
#     logging.info("*********************************************************************************************")
#     time.sleep(sleep_duration)

In [7]:
page = response.text
doc = soup(page, "html.parser")
items = [element for element in doc.find_all('li', {"class": "s-item"})]
len(items)

201

In [34]:
for k,item in enumerate(items[1:]):
    # print(item)
    url = item.find('a', {'class': 's-item__link'}).attrs['href']
    item_id = url.split("?hash")[0]
    item_id = item_id.split("/")[-1]
    item_id = item_id.split("?")[0]
    #TODO: correct for wrong case
    title = item.find('h3', {'class': 's-item__title'}).text
    item_condition = item.find('span', {'class': 'SECONDARY_INFO'})
    if item_condition:
        item_condition = item_condition.text
    buyitnow = item.find('span', {'class': 's-item__dynamic s-item__buyItNowOption'})
    if buyitnow:
        buyitnow = buyitnow.text
    shipping_price = item.find('span', {'class': 's-item__shipping s-item__logisticsCost'})
    shipping = False
    if shipping_price:
        shipping_price = shipping_price.text
        shipping = True
    image_url = item.find('img', {'class': "s-item__image-img"}).attrs['src']
    
    highest_price = item.find('span', {'class': 's-item__price'}).text
    nb_bids = item.find('span', {'class': 's-item__bids s-item__bidCount'}).text
    update_time = datetime.now()
    remaining_time = item.find('span', {'class': 's-item__time-left'}).text
    # TODO: calculate endtime
    # end_time = update_time + remaining_time
    
    
    print(url)
    print(image_url)
    print(item_id)
    print(title)
    print(item_condition)
    print(highest_price)
    print(shipping_price)
    print(nb_bids)
    print(remaining_time)
    print(buyitnow)
    # if buyitnow:
    #     print("\n\n\n\n\n\n\n")
    print("********************************************************************************")

https://www.ebay.de/itm/Apple-iPhone-7-32GB-Gold-Ohne-Simlock-A1778-GSM/184415707172?epid=225203755&hash=item2af0086024%3Ag%3AW7kAAOSwdO9fQuOb&LH_Auction=1
https://i.ebayimg.com/thumbs/images/g/W7kAAOSwdO9fQuOb/s-l140.webp
184415707172
Apple iPhone 7 - 32GB - Gold (Ohne Simlock) A1778 (GSM)
Gebraucht
EUR 142,00
+EUR 4,99 Versand
20 Gebote
29Sek
None
********************************************************************************
https://www.ebay.de/itm/Apple-ipad-air-Schutzfolie-Dispo-crystalclear-inkl-Rakel-und-Anleitung-1-Stuck/293703407184?hash=item4462165650%3Ag%3Aut4AAOSwxxJenDbk&LH_Auction=1
https://i.ebayimg.com/thumbs/images/g/ut4AAOSwxxJenDbk/s-l140.webp
293703407184
Apple ipad air Schutzfolie Dispo crystalclear inkl. Rakel und Anleitung -1 Stück
Brandneu
EUR 1,00
+EUR 4,99 Versand
0 Gebote
1 Min
Sofort-Kaufen
********************************************************************************
https://www.ebay.de/itm/Apple-iPhone-11-256GB-Schwarz-black-unlocked-A2221-Smartphone-NE

## For Item search query

In [62]:
# item_url = "https://www.ebay.de/itm/Apple-MacBook-Air-11-6-Zoll-Juni-2013/143699259235?hash=item217524af63%3Ag%3AlOcAAOSwfEVfRSRF&LH_Auction=1"
item_url = "https://www.ebay.de/itm/HP-OfficeJet-Pro-8210-Tintenstrahldrucker-Drucker-LAN-WLAN-Duplex-HP-I/264839280978?epid=2254414883&hash=item3da9a6b952%3Ag%3AKLoAAOSwbW9fP3MZ&LH_Auction=1"

In [63]:
response = requests.get(item_url, headers=headers)
with open("item.html", "wb") as f: 
    # Writing data to a file 
    f.write(response.content) 

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.ebay.de:443
DEBUG:urllib3.connectionpool:https://www.ebay.de:443 "GET /itm/HP-OfficeJet-Pro-8210-Tintenstrahldrucker-Drucker-LAN-WLAN-Duplex-HP-I/264839280978?epid=2254414883&hash=item3da9a6b952%3Ag%3AKLoAAOSwbW9fP3MZ&LH_Auction=1 HTTP/1.1" 200 None


In [64]:
page = response.text
doc = soup(page, "html.parser")

In [65]:
payments = doc.find('div', {'class': 'app-payments-btf-wrapper__paymentmethod'})
payments = payments.find_all('span')[::2]
payments = [p.text for p in payments]
payments


['Kreditkarte',
 'Barzahlung bei Abholung',
 'Kreditkarte (bitte Verkäufer kontaktieren)',
 'Überweisung']

In [66]:
location = doc.find('div', {'class': 'app-location-wrapper'})
location = location.find('div', {'class': 'cc-textblock'}).text
location

'Maßbach, Deutschland'

In [67]:
info = doc.find('div', {'class': 'app-chevron-wrapper__wrapper row'})
info = info.find_all('span')[::2]
info = [i.text for i in info]
info = [info[i*2] + ": " + info[i*2+1] for i in range(len(info) // 2)]
info

['Artikelzustand: Neu: Sonstige (siehe Artikelbeschreibung)',
 'Anmerkungen des Verkäufers: Habe den Drucker am 13.8.20 gekauft, ist also neuwertig und hat noch Garantie bis 13.8.21',
 'Stückzahl: 1 verfügbar',
 'Technologie: Inkjet',
 'Herstellernummer: D9L63A#A81',
 'Besonderheiten: Duplex, Netzwerkfähig',
 'Model: D9L63A']

In [68]:
seller_name = doc.find('span', {'class': 'app-sellerpresence__sellername'}).text
seller_name

'44190'

In [75]:
image_urls = doc.find_all('script')
image_urls = image_urls[-1].contents[-1]
m = re.findall('"originalSize":{"height":400,"width":400},"URL":"https://i.ebayimg.com/images(.+?)"}', image_urls)
image_urls = ["https://i.ebayimg.com/images" + m[i] for i in range(len(m)//2)]
image_urls

['https://i.ebayimg.com/images/g/KLoAAOSwbW9fP3MZ/s-l400.png']

In [76]:
nb_images = len(image_urls)
nb_images

1

In [77]:
description = doc.find('div', {"class": 'app-item-description__body cc-stdmargin__top--half'})
description = description.find('span', {"class": "app-item-description__body--text"})
description = description.text
description

'Drucktechnik: Tintenstrahl. Netzwerkfähiger Tintenstrahldrucker. Druckertyp: Einzelplatzdrucker. Druckgeschwindigkeit bis zu 22 S./Min. Duplexdruckgeschwindigkeit S/W mit bis zu 12 Seiten/Minute. Randloser Druck.'

In [78]:
detail_description = doc.find_all('script')
detail_description = detail_description[-1].contents[-1]
detail_description
m = re.findall('"URL":"http://vi.raptor.ebaydesc.com/ws/eBayISAPI.dll(.+?)"}}', detail_description)
if m:
    detail_description = "http://vi.raptor.ebaydesc.com/ws/eBayISAPI.dll" + m[0]
    try:
        imgkit.from_url(detail_description, 'detail_description_{}.jpg'.format(1))
    except:
        pass

Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               


## For Bid number query

In [79]:
search_url = "https://www.ebay.de/bfl/viewbids/{}".format(item_id)
# search_url = "https://www.ebay.de/bfl/viewbids/{}".format(264839280978)
search_url

'https://www.ebay.de/bfl/viewbids/264839280978'

In [80]:
response = requests.get(search_url, headers=headers)
with open("bids.html", "wb") as f: 
    # Writing data to a file 
    f.write(response.content) 

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.ebay.de:443
DEBUG:urllib3.connectionpool:https://www.ebay.de:443 "GET /bfl/viewbids/264839280978 HTTP/1.1" 200 None


In [81]:
page = response.text
doc = soup(page, "html.parser")

In [82]:
# overall = doc.find('div', {'class': 'app-mobile-container-header_wrapper'})
overall = doc.find_all('div', {'class': 'ui-label-value-list__value'})
overall = [div.text for div in overall]
nb_bids = overall[0]
print(nb_bids)
nb_bidders = overall[1]
print(nb_bidders)

1
1


In [83]:
bid_history = doc.find('div', {'class': 'app-bid-history__container'})
bid_history = bid_history.find_all('span')
bid_history = bid_history[::2]
bid_history = [span.text for span in bid_history]
start_price = bid_history[-1]
print(start_price)
bid_history = [{"bidder": bid_history[i*3], \
                "price": bid_history[i*3+1], \
                "datetime": bid_history[i*3+2]} \
                for i in range(len(bid_history)//3)]
print(bid_history)
highest_price = bid_history[0]['price']
print(highest_price)

EUR 60,00
[{'bidder': '2***9', 'price': 'EUR 60,00', 'datetime': '23 Aug 2020, 19:29:12 MESZ'}]
EUR 60,00


## For ebay seller

In [84]:
user_url = "https://www.ebay.de/usr/{}".format(seller_name)
user_url

'https://www.ebay.de/usr/44190'

In [85]:
response = requests.get(user_url, headers=headers)
with open("user.html", "wb") as f: 
    # Writing data to a file 
    f.write(response.content) 

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.ebay.de:443
DEBUG:urllib3.connectionpool:https://www.ebay.de:443 "GET /usr/44190 HTTP/1.1" 200 14089


In [86]:
page = response.text
doc = soup(page, "html.parser")

In [87]:
feedback_score = doc.find('div',{'class':'mbg'})
feedback_score = feedback_score.find_all('a')
feedback_score = feedback_score[-1]
feedback_score = feedback_score.contents[-1]
feedback_score

'388'

In [88]:
feedback_percentage = doc.find('div', {'class': 'perctg'})
feedback_percentage = feedback_percentage.text.strip()
feedback_percentage

'100% positive Bewertungen'

In [89]:
registered_since = doc.find('h2', {'class': 'bio inline_value'})
registered_since = registered_since.text.strip()
registered_since
# TODO: seperate location and time

'Standort: Deutschland, 44190 ist eBay-Mitglied seit 13. Jan. 2002'

In [90]:
feedback_ratings = doc.find('div', {'id': 'feedback_ratings'})
feedback_ratings = feedback_ratings.find_all('span', {'class': 'num'})
feedback_ratings = [f.text for f in feedback_ratings]
feedback_ratings

['19', '0', '0']